## Crawler (selenium)

In [1]:
from selenium import webdriver
import pandas as pd
import re

In [2]:
driver = webdriver.Chrome('chromedriver')
driver.get('https://music.yandex.ru/non-music/category/scienceandeducation_science/albums')

In [3]:
podcasts = pd.DataFrame(columns=['Header', 'Likes', 'Link'])

In [5]:
def scrap_page():
    
    global podcasts
    
    header_elements = driver.find_elements_by_xpath('//div[@class="album__bottom"]/div[@class="album__bottom-right"]/div[@class="album__title deco-typo typo-main"]/a[@class="d-link deco-link album__caption"]/span[@class="no-breakable-explicit-title"]/span[@class="no-breakable-explicit-title__text"]')
    likes_elements = driver.find_elements_by_xpath('//div[@class="album__bottom"]/div[@class="album__bottom-right"]/div[@class="album__podcast-info typo-add deco-typo-secondary"]')
    link_elements = driver.find_elements_by_xpath("//*[contains(@href,'/album/')]")
    
    for i in range(len(header_elements)):
       
        pod = {'Header': header_elements[i].text,
               'Likes': likes_elements[i].text,
               'Link': link_elements[i].get_attribute('href')}

                   
        podcasts = podcasts.append(pod, ignore_index=True)

In [6]:
scrap_page()
podcasts

C:\Users\Daria\AppData\Local\Temp/ipykernel_11660/2194243566.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  header_elements = driver.find_elements_by_xpath('//div[@class="album__bottom"]/div[@class="album__bottom-right"]/div[@class="album__title deco-typo typo-main"]/a[@class="d-link deco-link album__caption"]/span[@class="no-breakable-explicit-title"]/span[@class="no-breakable-explicit-title__text"]')
C:\Users\Daria\AppData\Local\Temp/ipykernel_11660/2194243566.py:6: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  likes_elements = driver.find_elements_by_xpath('//div[@class="album__bottom"]/div[@class="album__bottom-right"]/div[@class="album__podcast-info typo-add deco-typo-secondary"]')
C:\Users\Daria\AppData\Local\Temp/ipykernel_11660/2194243566.py:7: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  link_elements = drive

,Header,Likes,Link
0,Доминируй. Властвуй. Унижай,27 286,https://music.yandex.ru/album/12098244
1,Хакни мозг,16 007,https://music.yandex.ru/album/19760408
2,TED Лучшее,14 954,https://music.yandex.ru/album/21508580
3,Почему мы еще живы,20 751,https://music.yandex.ru/album/15741932
4,Кот ученый,2 300,https://music.yandex.ru/album/18408233
5,Миражи,6 166,https://music.yandex.ru/album/12728995
6,Вопрос учёному,7 229,https://music.yandex.ru/album/6964717
7,АНТРОПОГЕНЕЗ.РУ,3 882,https://music.yandex.ru/album/13875854
8,TED Лучшее: Общество и культура,5 147,https://music.yandex.ru/album/21327092
9,TED Лучшее: Бизнес,3 776,https://music.yandex.ru/album/21326972


In [24]:
def metadata_via_links(df_with_links):
    
    descriptions, n_of_episodes, years = [], [], []
    
    for link in df_with_links['Link']:
        driver.get(link)
        
        # descriptions (append None if there is no description)
        try:
            read_more = driver.find_element_by_xpath('//span[@class="d-link deco-link d-collapsable-description__read-more d-link_marked deco-link_marked"]').click()

        except:
            pass
    
        try:
            description = driver.find_element_by_xpath('//div[@class="d-collapsable-description typo deco-link-parent deco-link-parent_marked page-album__desc"]')
            descriptions.append(description.text)
    
        except:
            descriptions.append(None)
        
        # year (append None if there is no year)
        try:
            year = driver.find_element_by_xpath('//div[@class="d-album-summary__group d-album-summary__item typo-disabled"]/span[@class="typo deco-typo-secondary"]')
            years.append(year.text)

        except:
            years.append(None)
        
        # number of episodes
        n_episodes = driver.find_element_by_xpath('//div[@class="d-subhead page-album__episode-count"]/span[@class="d-link deco-link d-subhead__wrapper d-link_no-hover-color deco-link_no-hover-color d-link_no-hover"]/div[@class="d-subhead__title typo typo-h2_bold"]/h2[@class="d-subhead__title-main"]/span[@class="d-link deco-link d-subhead__title-main d-link_no-hover-color deco-link_no-hover-color d-link_no-hover"]')
        n = re.match(r'\d+', n_episodes.text).group(0)
        n_of_episodes.append(n)
        
        print(f'{podcasts.index[podcasts["Link"] == link][0] + 1} out of {podcasts.shape[0]} done')
        

    
    df_with_links['Number of Episodes'] = n_of_episodes
    df_with_links['Year'] = years
    df_with_links['Description'] = descriptions


In [25]:
metadata_via_links(podcasts)
podcasts

1 out of 397 done
2 out of 397 done
3 out of 397 done
4 out of 397 done
5 out of 397 done
6 out of 397 done
7 out of 397 done
8 out of 397 done
9 out of 397 done
10 out of 397 done
11 out of 397 done
12 out of 397 done
13 out of 397 done
14 out of 397 done
15 out of 397 done
16 out of 397 done
17 out of 397 done
18 out of 397 done
19 out of 397 done
20 out of 397 done
21 out of 397 done
22 out of 397 done
23 out of 397 done
24 out of 397 done
25 out of 397 done
26 out of 397 done
27 out of 397 done
28 out of 397 done
29 out of 397 done
30 out of 397 done
31 out of 397 done
32 out of 397 done
33 out of 397 done
34 out of 397 done
35 out of 397 done
36 out of 397 done
37 out of 397 done
38 out of 397 done
39 out of 397 done
40 out of 397 done
41 out of 397 done
42 out of 397 done
43 out of 397 done
44 out of 397 done
45 out of 397 done
46 out of 397 done
47 out of 397 done
48 out of 397 done
49 out of 397 done
50 out of 397 done
51 out of 397 done
52 out of 397 done
53 out of 397 done
54

,Header,Likes,Link,Number of Episodes,Year,Description
0,Доминируй. Властвуй. Унижай,17 023,https://music.yandex.ru/album/12098244,17,None,"Ответы на вопросы, которые вы не задавали...\n..."
1,"Виват, история!",12 782,https://music.yandex.ru/album/7121797,394,None,Самые яркие страницы истории отечественной и н...
2,SCP Foundation,4 473,https://music.yandex.ru/album/10431351,415,None,Что мы знаем о вселенной SCP Foundation?\nФонд...
3,Три Богатыря,478,https://music.yandex.ru/album/12044498,15,None,"«Преданья старины глубокой» (мифы, легенды и с..."
4,Голый землекоп,21 704,https://music.yandex.ru/album/10330389,41,None,Биолог и журналист Илья Колмановский рассказыв...
...,...,...,...,...,...,...
392,Сова говорит…,38,https://music.yandex.ru/album/8469435,4,None,"Делюсь опытом Frontend разработки, размышляю о..."
393,Born to science podcast,270,https://music.yandex.ru/album/7269164,4,None,None
394,CO₂-free подкаст о меняющемся климате,247,https://music.yandex.ru/album/8291093,14,None,"Подкаст, в котором мы будем разбираться что та..."
395,Лингвоед (Lingvoed),72,https://music.yandex.ru/album/8063756,6,None,"Лингвоед — подкаст для тех, кто любит русский ..."


In [26]:
podcasts.to_excel('C:/Users/User/Downloads/podcasts_with_descr.xlsx', encoding='utf-8-sig')

In [31]:
podcasts_with_descr = pd.read_excel('podcasts_with_descr.xlsx')
podcasts_with_descr['Number of Episodes'] = podcasts_with_descr['Number of Episodes'].astype('int64')
podcasts_with_descr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          397 non-null    int64  
 1   Header              397 non-null    object 
 2   Likes               397 non-null    object 
 3   Link                397 non-null    object 
 4   Number of Episodes  397 non-null    int64  
 5   Year                18 non-null     float64
 6   Description         379 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 21.8+ KB


# Drafts

In [20]:
def get_description(by_link):  

    driver.get(by_link)
    driver.implicitly_wait(3)
    
    # для кнопки "читать полностью"
    try:
        read_more = driver.find_element_by_xpath('//span[@class="d-link deco-link d-collapsable-description__read-more d-link_marked deco-link_marked"]').click()

    except:
        pass
    
    # на случай, если описание отсутствует
    try:
        description_element = driver.find_elements_by_xpath('//div[@class="d-collapsable-description typo deco-link-parent deco-link-parent_marked page-album__desc"]')
        return description_element[0].text
    
    except IndexError:
        return ''

In [27]:
descriptions = []


for link in podcasts['Link']:
    if len(descriptions) < 5:
        descriptions.append(get_description(link))

podcasts['Description'][:5] = descriptions

In [28]:
podcasts

,Header,Likes,Link,Description
0,Доминируй. Властвуй. Унижай,14 112,https://music.yandex.ru/album/12098244,"Ответы на вопросы, которые вы не задавали...\n..."
1,"Виват, история!",11 745,https://music.yandex.ru/album/7121797,Самые яркие страницы истории отечественной и н...
2,SCP Foundation,4 049,https://music.yandex.ru/album/10431351,Что мы знаем о вселенной SCP Foundation?\nФонд...
3,Миражи,4 241,https://music.yandex.ru/album/12728995,Подкаст об особенностях восприятия и когнитивн...
4,Три Богатыря,420,https://music.yandex.ru/album/12044498,"«Преданья старины глубокой» (мифы, легенды и с..."
...,...,...,...,...
362,Наружу,76,https://music.yandex.ru/album/7106268,NaN
363,Теорема Лаговского,287,https://music.yandex.ru/album/7098685,NaN
364,Око Хокинга,565,https://music.yandex.ru/album/6951090,NaN
365,Научно-популярное шоу «Опытные на кухне»,717,https://music.yandex.ru/album/6410467,NaN


In [44]:
# podcasts.to_csv('C:/Users/User/Downloads/podcasts_with_descr.csv', encoding='utf-8-sig')
# podcasts.to_excel("output.xlsx")
podcasts_with_descr = pd.read_csv('podcasts_with_descr.csv')
podcasts_with_descr

,Unnamed: 0,Header,Likes,Link,Description
0,0,Доминируй. Властвуй. Унижай,14 009,https://music.yandex.ru/album/12098244,"Ответы на вопросы, которые вы не задавали...\n..."
1,1,"Виват, история!",11 723,https://music.yandex.ru/album/7121797,Самые яркие страницы истории отечественной и н...
2,2,SCP Foundation,4 041,https://music.yandex.ru/album/10431351,Что мы знаем о вселенной SCP Foundation?\nФонд...
3,3,Миражи,4 219,https://music.yandex.ru/album/12728995,Подкаст об особенностях восприятия и когнитивн...
4,4,Три Богатыря,419,https://music.yandex.ru/album/12044498,"«Преданья старины глубокой» (мифы, легенды и с..."
...,...,...,...,...,...
361,361,Born to science podcast,263,https://music.yandex.ru/album/7269164,NaN
362,362,Наружу,75,https://music.yandex.ru/album/7106268,У реальности много слоев. Наша жизнь протекает...
363,363,Теорема Лаговского,287,https://music.yandex.ru/album/7098685,"Мировые открытия в науке и технике, важные соб..."
364,364,Око Хокинга,563,https://music.yandex.ru/album/6951090,"Око Хокинга — это научное сообщество Саратова,..."
